In [1]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt

2022-06-08 22:50:23.542531: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-08 22:50:23.545726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-08 22:50:23.545737: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
df = pd.read_json('files/embeddings.json')

In [3]:
df['vector'] = df['CarNewsTitle'] + df['Highlights'] + df['Description']

In [4]:
df.drop(['CarNewsTitle', 'Highlights', 'Description'], axis=1, inplace=True)

In [5]:
df = pd.merge(df, df, how='cross')
df = df[['CarNewsId_x', 'CarNewsId_y', 'vector_x', 'vector_y']]
df.rename(columns={'CarNewsId_x':'id1', 'CarNewsId_y':'id2', 'vector_x':'vector_1', 'vector_y':'vector_2'}, inplace=True)

In [6]:
random100_s1 = [3,8,34,43,44,45,50,53,56,59,62,65,67,69,71,91,118,123,128,134,137,138,176,188,194,196,201,207,214,218,229,243,248,257,267,268,276,298,301,315,316,321,325,329,333,349,357,379,389,411,428,452,469,474,490,514,526,528,541,556,562,565,572,575,580,583,600,622,624,630,640,648,682,683,706,707,717,720,729,732,738,739,743,753,760,761,762,767,777,818,837,838,845,846,848,852,856,862,874,876]
random100_s2 = [22,25,27,42,49,54,55,76,93,94,96,97,120,150,163,173,186,189,195,204,208,209,212,221,227,242,251,255,262,265,266,272,287,302,305,310,318,331,345,372,373,383,391,398,408,424,433,434,435,454,456,459,473,475,479,480,496,501,502,510,540,545,560,574,590,606,612,623,669,681,689,695,721,726,727,731,737,756,766,779,786,792,794,798,799,802,815,817,821,823,828,829,831,842,850,858,859,866,867,873]

In [7]:
df_s1 = pd.read_csv('files/top15_simdesc_cleanedsample1.csv')
df_s2 = pd.read_csv('files/top15_simdesc_cleanedsample2.csv')
df_s = pd.concat([df_s1, df_s2], ignore_index=True, sort=False)

In [8]:
df2 = pd.merge(df, df_s, on=['id1','id2'])
df2 = df2[['id1','id2','vector_1','vector_2','classification']]

In [9]:
df2

,id1,id2,vector_1,vector_2,classification
0,28032,28214,"[[-0.22375138100000003, 0.4476603866, 0.236245...","[[0.307490617, 0.0315352827, 0.4327310026, 0.7...",0
1,28032,28221,"[[-0.22375138100000003, 0.4476603866, 0.236245...","[[-0.6250770688, 0.4015112817, 0.8460243940000...",0
2,28032,28314,"[[-0.22375138100000003, 0.4476603866, 0.236245...","[[-0.1413342953, 1.1729570627, 0.3858661652, -...",0
3,28032,28315,"[[-0.22375138100000003, 0.4476603866, 0.236245...","[[-0.1413342953, 1.1729570627, 0.3858661652, -...",0
4,28032,28376,"[[-0.22375138100000003, 0.4476603866, 0.236245...","[[1.1202390194, 0.7119821906, -0.0169125833, -...",0
...,...,...,...,...,...
2995,29087,28955,"[[-1.3961848021, 0.39991816880000003, -0.68192...","[[-1.0581085682, 0.5650228858, 0.3123189211000...",0
2996,29087,28964,"[[-1.3961848021, 0.39991816880000003, -0.68192...","[[-1.3613573313, 0.8344203830000001, 0.1985223...",0
2997,29087,29028,"[[-1.3961848021, 0.39991816880000003, -0.68192...","[[0.044806093000000005, 0.7397106886, -0.43723...",0
2998,29087,29037,"[[-1.3961848021, 0.39991816880000003, -0.68192...","[[-1.6479408741000001, 0.4668063223, -0.623096...",0


In [10]:
X = df2.drop(['id1','id2','classification'], axis=1).values
y = df2.iloc[:,-1:].values
_, m = X.shape
_, n = y.shape

In [12]:
n

1

In [13]:
lr = 0.05
epochs = 50
display_step = 10

In [14]:
X = tf.placeholder(tf.float32, shape=[None,m], name='Input')
Y = tf.placeholder(tf.float32, shape=[None,n], name='Output')

W = tf.Variable(tf.zeros([m,n]))
b = tf.Variable(tf.zeros([n]))

In [15]:
def sigmoid(z):
    return 1/(1+np.exp(-z))
Y_hat = tf.nn.sigmoid(tf.add(tf.matmul(X,W),b))
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Y_hat,labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(cost)

In [16]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    cost_history, accuracy_history = [], []
    for epoch in range(epochs):
        cost_per_epoch = 0
        _, c = sess.run([optimizer,cost], feed_dict={X:X,Y:y})
        correct_prediction = tf.equal(tf.round(Y_hat), Y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        cost_history.append(c)
        accuracy_history.append(accuracy.eval({X:X,Y:y})*100)
        if (epoch+1)% display_step == 0 and epoch != 0:
            print('Epoch: {4.0f} - Cost: {0.5f} - Acc: {0.5f}'.format(epoch+1,c,accuracy_history[-1]))
    Weight = sess.run(W)
    Bias = sess.run(b)
    correct_prediction = tf.equal(tf.arg_max(Y_hat,1),tf.arg_max(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    print('\nAccuracy:', accuracy_history[-1], '%')

2022-06-08 22:52:16.803321: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-08 22:52:16.803346: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-08 22:52:16.803367: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cssaivishnu): /proc/driver/nvidia/version does not exist
2022-06-08 22:52:16.803616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-08 22:52:16.805167: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354]

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles. For reference, the tensor object was Tensor("Input:0", shape=(?, 2), dtype=float32) which was passed to the argument `feed_dict` with key Tensor("Input:0", shape=(?, 2), dtype=float32).

In [ ]:
df2

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df2.drop(['id1','id2','classification'], axis=1).values
y = df2.iloc[:,-1:].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [ ]:
_, m = X_train.shape
_, n = y_train.shape

In [ ]:
lr = 0.05
epochs = 50
display_step = 10

In [ ]:
X = tf.placeholder(tf.float32, shape=[None,m], name='Input')
Y = tf.placeholder(tf.float32, shape=[None,n], name='Output')

W = tf.Variable(tf.zeros([m,n]))
b = tf.Variable(tf.zeros([n]))

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))
Y_hat = tf.nn.sigmoid(tf.add(tf.matmul(X,W),b))
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Y_hat,labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(cost)

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    cost_history, accuracy_history = [], []
    for epoch in range(epochs):
        cost_per_epoch = 0
#        sess.run(optimizer, feed_dict={X:X_train,Y:y_train})
        _, c = sess.run([cost,optimizer], feed_dict={X:X_train,Y:y_train})
        correct_prediction = tf.equal(tf.round(Y_hat), Y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        cost_history.append(c)
        accuracy_history.append(accuracy.eval({X:X_train,Y:y_train})*100)
        if (epoch+1)% display_step == 0 and epoch != 0:
            print('Epoch: {4.0f} - Cost: {0.5f} - Acc: {0.5f}'.format(epoch+1,c,accuracy_history[-1]))
    Weight = sess.run(W)
    Bias = sess.run(b)
    correct_prediction = tf.equal(tf.arg_max(Y_hat,1),tf.arg_max(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    print('\nAccuracy:', accuracy_history[-1], '%')